<a href="https://colab.research.google.com/github/dhy02094/STUDY_TH/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# 압축 풀기

# import zipfile
# DATA_IN_PATH = '/content/drive/MyDrive/Kt/data_in/'
# file_list = ['labeledTrainData.tsv.zip', 'unlabeledTrainData.tsv.zip', 'testData.tsv.zip']

# for file in file_list:
#     zipRef = zipfile.ZipFile(DATA_IN_PATH + file, 'r')
#     zipRef.extractall(DATA_IN_PATH)
#     zipRef.close()

파일 크기 : 
labeledTrainData.tsv          33.56MB
unlabeledTrainData.tsv        67.28MB
testData.tsv                  32.72MB


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [12]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

print("파일 크기 : ")
for file in os.listdir(DATA_IN_PATH):
    if 'tsv' in file and 'zip' not in file:
        print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000, 2)) + 'MB')

train_data = pd.read_csv( DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', quoting=3)
train_data.head()

파일 크기 : 
labeledTrainData.tsv          33.56MB
unlabeledTrainData.tsv        67.28MB
testData.tsv                  32.72MB


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [13]:
print('전체 학습데이터의 개수 : {}'.format(len(train_data)))

전체 학습데이터의 개수 : 25000


In [14]:
train_length = train_data['review'].apply(len)
train_length.head()

0    2304
1     948
2    2451
3    2247
4    2233
Name: review, dtype: int64

step2 preprocessing

In [15]:
import re 
import json 
import pandas as pd 
import numpy as np 
import nltk 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer



In [21]:
DATA_IN_PATH = '/content/drive/MyDrive/Kt/data_in/'
train_data = pd.read_csv( DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', quoting=3)

review = train_data['review'][0] #첫 번째 리뷰를 가져옴
review_text = BeautifulSoup(review,'html5lib').get_text()   #HTML 태그 제거
review_text = re.sub("[^a-zA-Z]", " ", review_text)        #영어 문자를 제외한 나머지는 모두 공백으로 바꿈
print(review_text)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))    # 영어 불용어 집합 구성

review_text = review_text.lower()
words = review_text.split()                     # 소문자로 바꿔서 단어 리스트로 만듦
words = [w for w in words if not w in stop_words]
print(words)
clean_review = ' '.join(words)                 # 단어 리스트들을 다시 하나의 글로
print(clean_review)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [24]:
def preprocessing(review, remove_stopwords=False):
    review_text = BeautifulSoup(review, "html5lib").get_text()
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    review_text = review_text.lower()
    if remove_stopwords:
        words = review_text.split()
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        review_text = ' '.join(words)
    return review_text

clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))
clean_train_df = pd.DataFrame({'review' : clean_train_reviews,
                               'sentiment' : train_data['sentiment']})


In [25]:
clean_train_df

,review,sentiment
0,stuff going moment mj started listening music ...,1
1,classic war worlds timothy hines entertaining ...,1
2,film starts manager nicholas bell giving welco...,0
3,must assumed praised film greatest filmed oper...,0
4,superbly trashy wondrously unpretentious explo...,1
...,...,...
24995,seems like consideration gone imdb reviews fil...,0
24996,believe made film completely unnecessary first...,0
24997,guy loser get girls needs build picked stronge...,0
24998,minute documentary bu uel made early one spain...,0


In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)
print(text_sequences[0])

MAX_SEQUENCE_LENGTH = 174
train_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
print('Train Data:', train_inputs.shape)

train_labels = np.array(train_data['sentiment'])
print('Label:', train_labels.shape)



[404, 70, 419, 8815, 506, 2456, 115, 54, 873, 516, 178, 18686, 178, 11242, 165, 78, 14, 662, 2457, 117, 92, 10, 499, 4074, 165, 22, 210, 581, 2333, 1194, 11242, 71, 4826, 71, 635, 2, 253, 70, 11, 302, 1663, 486, 1144, 3265, 8815, 411, 793, 3342, 17, 441, 600, 1500, 15, 4424, 1851, 998, 146, 342, 1442, 743, 2424, 4, 8815, 418, 70, 637, 69, 237, 94, 541, 8815, 26055, 26056, 120, 1, 8815, 323, 8, 47, 20, 323, 167, 10, 207, 633, 635, 2, 116, 291, 382, 121, 15535, 3315, 1501, 574, 734, 10013, 923, 11578, 822, 1239, 1408, 360, 8815, 221, 15, 576, 8815, 22224, 2274, 13426, 734, 10013, 27, 28606, 340, 16, 41, 18687, 1500, 388, 11243, 165, 3962, 8815, 115, 627, 499, 79, 4, 8815, 1430, 380, 2163, 114, 1919, 2503, 574, 17, 60, 100, 4875, 5100, 260, 1268, 26057, 15, 574, 493, 744, 637, 631, 3, 394, 164, 446, 114, 615, 3266, 1160, 684, 48, 1175, 224, 1, 16, 4, 8815, 3, 507, 62, 25, 16, 640, 133, 231, 95, 7426, 600, 3439, 8815, 37248, 1864, 1, 128, 342, 1442, 247, 3, 865, 16, 42, 1487, 997, 2333, 12

In [32]:
import os 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000)
X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)
print('Accuracy: %f' % lgs.score(X_test, y_test))



(20000, 5000) (5000, 5000) (20000,) (5000,)
Accuracy: 0.869600


In [36]:
from keras.models import Sequential
from keras.layers import Dense
X_train1 = X_train.toarray()
X_test1 = X_test.toarray()
# Model, Cost, Train
model = Sequential()
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train1, y_train, epochs=200, verbose=1)

#Testing
_, accuracy = model.evaluate(X_test1, y_test)
print('Accuracy: ', accuracy)
model.summary()

Epoch 1/200
625/625 [==============================] - 13s 20ms/step - loss: 0.6924 - accuracy: 0.5252
Epoch 2/200
625/625 [==============================] - 10s 16ms/step - loss: 0.6902 - accuracy: 0.6248
Epoch 3/200
625/625 [==============================] - 9s 15ms/step - loss: 0.6880 - accuracy: 0.6676
Epoch 4/200
625/625 [==============================] - 9s 15ms/step - loss: 0.6854 - accuracy: 0.6801
Epoch 5/200
625/625 [==============================] - 9s 15ms/step - loss: 0.6825 - accuracy: 0.6913
Epoch 6/200
625/625 [==============================] - 9s 15ms/step - loss: 0.6791 - accuracy: 0.7477
Epoch 7/200
625/625 [==============================] - 10s 15ms/step - loss: 0.6749 - accuracy: 0.7569
Epoch 8/200
625/625 [==============================] - 11s 17ms/step - loss: 0.6699 - accuracy: 0.7508
Epoch 9/200
625/625 [==============================] - 9s 15ms/step - loss: 0.6643 - accuracy: 0.7612
Epoch 10/200
625/625 [==============================] - 10s 15ms/step - loss: 